In [2]:
clear

X=rand(3,100);

X=X./repmat(sqrt(sum(X.*X)),size(X,1),1);

Y=rand(3,1000);

Y=Y./repmat(sqrt(sum(Y.*Y)),size(Y,1),1);

A=X'*Y;

B=X'*X;

I=randi([1,100],1,10);

J=randi([1,1000],1,10);

index=sub2ind([100,1000],I,J);

C=A(index);

A(index)=nan;

In [6]:
% for idx = 1:length(index)
%     [row, col] = ind2sub([100, 1000], index(idx));
    
% end

In [7]:
[num_users, num_items] = size(A);
A_est = A;

for idx = 1:length(index)
    i = I(idx);
    j = J(idx);
    
    known_users = find(~isnan(A(:, j)));
    known_users(known_users == i) = [];
    
    if isempty(known_users)
        % 若没有任何用户对该商品有评分，则用全局平均值或0填充
        A_est(i,j) = nanmean(A(:));
        continue;
    end
    
    % 从B中取出用户i与已知用户的相似度
    sim_vec = B(i, known_users);
    
    % 已知评分
    rating_vec = A(known_users, j);
    
    % 对评分进行加权平均
    weight_sum = sum(sim_vec);
    if weight_sum == 0
        % 若相似度总和为0，则用全局平均值（或其他策略）填充
        A_est(i,j) = nanmean(A(:));
    else
        A_est(i,j) = sum(sim_vec .* rating_vec) / weight_sum;
    end
end

% 此时，A_est中已填充了先前的缺失值
% 可以检查填充精度（比如MSE）：
diff = A_est(index) - C;
mse = mean(diff.^2);
fprintf('填充缺失值后的MSE: %.4f\n', mse);

Unable to perform assignment because the size of the left side is 1-by-1 and the size of the right side is 1-by-99.

